## Ingest_lightsheet_data
The idea here is to use the google sheets API to ingest the information stored in the core facility clearing google sheets into the braincogs00 database. We will be making a new database table for each clearing protocol spreadsheet. These will be linked to the Experiments() table in the database by the experiment_id primary key.

In [4]:
import pickle
import os.path
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pandas as pd
import numpy as np
import datajoint as dj

## Google Spreadsheet API setup

We will be using the Google Sheets API: https://developers.google.com/sheets/api/quickstart/python

For details on how to set this up on your machine, see this notebook: [Microscope_management.ipynb](Microscope_management.ipynb).

In [2]:
SPREADSHEET_ID = '15NmKBIPfSSpjTFoHS6K2jREsbMZHueyQ5psub-bctjI' # The copy of the clearing spreadsheet, where I made some formatting changes to make it more consistent.
# Set the scope to be read only since we are not adding anything to the google sheet, just reading it
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

secrets_file = 'credentials.json' # Has to be called "credentials.json"  
assert os.path.exists(secrets_file)

In [5]:
def retrieve_google_sheet(spreadsheet_id,range_query):
    """
    ---PURPOSE---
    Gets the data from the range of cells in the google sheet specified.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=range_query).execute()
    values = result.get('values', [])

    return values

## Connect to the db

In [6]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.conn()

Please enter DataJoint username: ahoag
Please enter DataJoint password: ········
Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [7]:
# Need to port forward 3306 from jtb3-dev@pni.princeton.edu first (set this up externally on the machine)
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')

## Ingest the form responses into the User(), Request() and Request.Samples() tables

In [8]:
db_lightsheet.User()

username user in the lab,princeton_email


In [9]:
db_lightsheet.Request()

username user in the lab,request_name,requested_by user in the lab,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,subject_fullname,correspondence_email,description,species,number_of_samples,testing


In [10]:
# We are in readonly (see SCOPES) so we cannot destroy it by accident.
form_responses_range_query = 'Form Responses!A4:S54' # Syntax is Sheet_name!cell_start:cell_end, e.g. 
form_responses_values = retrieve_google_sheet(spreadsheet_id=SPREADSHEET_ID,range_query=form_responses_range_query)
df_form_responses = pd.DataFrame(form_responses_values)

In [11]:
def ingest_request_form(insert=False):
    user_insert_list = [{'username':'ahoag','princeton_email':'ahoag@princeton.edu'},
                        {'username':'zmd','princeton_email':'zmd@princeton.edu'},
                        {'username':'jduva','princeton_email':'jduva@princeton.edu'},
                        {'username':'kellyms','princeton_email':'kellyms@princeton.edu'}] # admins
    request_insert_list = []
    for row in df_form_responses.values.tolist():
        if not any(row): 
            # skip blank rows
            continue
        request_insert_dict = {}
        # handle email and user 
        email = row[-4]
        requested_by = ''
        if not email.endswith('princeton.edu'):
            if email == 'zahra.dhanerawala@gmail.com':
                email = 'marlies.oostland@princeton.edu'
                requested_by = 'zmd'
            elif email == 'emilyjanedennis@gmail.com':
                email = 'ejdennis@princeton.edu'
            else:
                continue
        email = email.lower()
        username = email.split('@')[0]
        user_insert_dict = {'username':username,'princeton_email':email}
        user_insert_list.append(user_insert_dict)
        request_insert_dict['username'] = username
        # handle request_name
        request_name = row[3][0:64].strip()
        request_name = '_'.join(request_name.split(' '))
        request_insert_dict['request_name'] = request_name
        # requested_by
        if requested_by != 'zmd':
            requested_by = username
        request_insert_dict['requested_by'] = requested_by
        # date_submitted and time_submitted
        date_input,time_input = row[0].split(' ')
        date_submitted = datetime.strptime(date_input,'%m/%d/%Y').strftime('%Y-%m-%d')
        time_submitted = time_input
        request_insert_dict['date_submitted'] = date_submitted
        request_insert_dict['time_submitted'] = time_submitted
        # labname
        labname = row[-1]
        if not labname:
            labname = 'not provided'
        request_insert_dict['labname'] = labname
        # subject_fullname
        request_insert_dict['subject_fullname'] = ''
        # correspondence email
        request_insert_dict['correspondence_email'] = email
        # description
        description = row[5][0:250]
        request_insert_dict['description'] = description
        # species
        species = row[1].lower()
        request_insert_dict['species'] = species
        # number_of_samples
        number_of_samples = row[4]
        if not number_of_samples:
            number_of_samples = 1
        else:
            number_of_samples = int(number_of_samples)
        request_insert_dict['number_of_samples'] = number_of_samples
        request_insert_list.append(request_insert_dict)
        
    if insert:
        db_lightsheet.User().insert(user_insert_list,skip_duplicates=True)
        db_lightsheet.Request().insert(request_insert_list,skip_duplicates=False)

ingest_request_form(insert=True)

In [12]:
db_lightsheet.Request() 

username user in the lab,request_name,requested_by user in the lab,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,subject_fullname,correspondence_email,description,species,number_of_samples,testing
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",afalkner,2019-11-18,17:22:11,Falkner,,afalkner@princeton.edu,"Fos induction, SW mice",mouse,1,0
apv2,20190313_IBL_DiI_1,apv2,2019-03-13,1:43:03,not provided,,apv2@princeton.edu,Detection of CM DiI,mouse,1,0
apv2,ibl_witten_04,apv2,2019-08-12,15:07:23,not provided,,apv2@princeton.edu,Detection of electrode tracks CM-DiI,mouse,1,0
ejdennis,"10-13_brains,_names_TBD",ejdennis,2019-08-06,17:12:15,not provided,,ejdennis@princeton.edu,atlas building,rat,1,0
ejdennis,201905_atlas00x_where_x=1:n,ejdennis,2019-05-21,19:12:45,not provided,,ejdennis@princeton.edu,"Brody lab rat atlas, iDisco + 1.3x",rat,1,0
ejdennis,20190606_atlas00x_where_x=11-20,ejdennis,2019-06-04,10:43:37,not provided,,ejdennis@princeton.edu,Brody lab rat atlas iDisco + 1.3x,rat,1,0
ejdennis,"E112,_E126,_E137",ejdennis,2020-03-11,12:03:39,Brody,,ejdennis@princeton.edu,"I will stain them using Yisi's vasculature staining protocol, the goal is to use these for aligning widefield imaging data of the dorsal cortical surface: E112 left hemisphere posterior to bregma, E126 left hemisphere posterior to bregma, E137 medial",rat,1,0
ejdennis,"K310_(CM-diI),_K315_(CM-diI),_K320,_K321,_K323,_K327,_K333,_K334",ejdennis,2020-02-16,14:11:46,Brody,,ejdennis@princeton.edu,for adding to the atlas with the improved clearing protocol AND four of these ten brains will have CM-DiI marking two locations in each brain,rat,1,0
ejdennis,VGAT-ReaChR_001,ejdennis,2020-01-27,20:12:13,Brody,,ejdennis@princeton.edu,we will coat a probe in CM-diI and perform an acute activation experiment. We will want to see where the probe was in the brain.,rat,1,0
ejdennis,"W118,_K292,_K293,_K295,_K301,_K302,_K303,_K304,_K305,_K306,_K307",ejdennis,2019-08-06,17:10:56,not provided,,ejdennis@princeton.edu,atlas building,rat,1,0
